In [115]:
from enum import Enum
import math
class Segment(Enum):
    a = 0b0000001
    b = 0b0000010
    c = 0b0000100
    d = 0b0001000
    e = 0b0010000
    f = 0b0100000
    g = 0b1000000

    #  aaaaa
    # b     c
    # b     c
    # b     c
    #  ddddd
    # e     f
    # e     f
    # e     f
    #  ggggg



class SegmentGroup():
    def __init__(self):
        self.value = 0; 


    def Encode(self, string):
        encoded = 0
        for i in range(len(string)):
            if string[i] == 'a':
                encoded |= Segment.a.value
            elif string[i] == 'b':
                encoded |= Segment.b.value
            elif string[i] == 'c':
                encoded |= Segment.c.value
            elif string[i] == 'd':
                encoded |= Segment.d.value
            elif string[i] == 'e':
                encoded |= Segment.e.value
            elif string[i] == 'f':
                encoded |= Segment.f.value
            elif string[i] == 'g':
                encoded |= Segment.g.value
        self.value = encoded
        self.flags = self.CountFlags()
        return encoded

        
        

    def __str__(self):
        str = ""
        for i in range(0,7):
            if self.value & (1 << i):
                str += Segment(1 << i).name
        return str
    
    def __repr__(self):
        str = ""
        for i in range(0,7):
            if self.value & (1 << i):
                str += Segment(1 << i).name
        return str
    
    def hasSegment(self, segment):
        return self & segment == segment
    
    # return count of flags in segment
    def CountFlags(self):
        count = 0
        for i in range(0,7):
            if self.value & (1 << i):
                count += 1
                
        return count
    
    # compare two segments
    def __eq__(self, other):
        return self.value == other.value

class SegmentBoard:
    def __init__(self, SegmentSignals, OutputSignals):
        self.signals = SegmentSignals
        self.output = OutputSignals
        self.decoder = [] *10
        self.value = 0
    
    
    def Decode(self):
        unknown = [x for x in self.signals]
        # and operate Signal Segment with 2 flags and 3 flags. this indicates the position for segment a
        self.decoderSegments = [[None, Segment.a], [None, Segment.b], [None, Segment.c], [None, Segment.d], [None, Segment.e], [None, Segment.f], [None, Segment.g]]
        display0 = None
        display1 = None
        display2 = None
        display3 = None
        display4 = None
        display5 = None
        display6 = None
        display7 = None
        display8 = None
        display9 = None
        for i in range(len(unknown)): 
            if self.signals[i].flags == 2:
                display1 = self.signals[i]
                unknown.remove(self.signals[i])
            elif self.signals[i].flags == 3:
                display7 = self.signals[i]
                unknown.remove(self.signals[i])
            elif self.signals[i].flags == 4:
                display4 = self.signals[i]
                unknown.remove(self.signals[i])
            elif self.signals[i].flags == 7:
                display8 = self.signals[i]
                unknown.remove(self.signals[i])
        

        self.decoderSegments[0][0] = Segment(display7.value ^ display1.value)
        i = 0
        while i < len(unknown):
            mask1 = SegmentGroup()
            mask1.value = (unknown[i].value & display4.value)

            mask2 = SegmentGroup()
            mask2.value = (unknown[i].value & display1.value)

            if mask1.CountFlags() == 3 and mask2.CountFlags() != 2 and unknown[i].flags == 6:
                #print("Found display 6")
                #print(f"{unknown[i]} ")
                #print(f"{unknown[i].value:7b} ")
                #print(f"{mask1.value:7b} ")
                display6 = unknown[i]
                unknown.remove(unknown[i])
            elif mask1.CountFlags() == 3 and mask2.CountFlags() != 2 and unknown[i].flags == 5:
                #print("Found display 5")
                #print(f"{unknown[i]} ")
                #print(f"{unknown[i].value:7b} ")
                #print(f"{mask1.value:7b} ")
                display5 = unknown[i]
                unknown.remove(unknown[i])
            else:
                i += 1
        
        i = 0
        while i < len(unknown):
            mask1 = SegmentGroup()
            mask1.value = (unknown[i].value & display4.value)

            mask2 = SegmentGroup()
            mask2.value = (unknown[i].value & display1.value)

            if mask1.CountFlags() == 3 and mask2.CountFlags() != 1 and unknown[i].flags == 5:
                #print("Found display 3")
                #print(f"{unknown[i]} ")
                #print(f"{unknown[i].value:7b} ")
                #print(f"{mask1.value:7b} ")
                display3 = unknown[i]
                unknown.remove(unknown[i])
            else:
                i += 1
        i = 0
        while i < len(unknown):
            mask1 = SegmentGroup()
            mask1.value = (unknown[i].value & display5.value)

            if mask1.CountFlags() == 5 and unknown[i].flags == 6:
                #print("Found display 9")
                #print(f"{unknown[i].value:7b} ")
                #print(f"{mask1.value:7b} ")
                display9 = unknown[i]
                unknown.remove(unknown[i])
            else:
                i += 1
        i = 0
        while i < len(unknown):
            if unknown[i].flags == 6:
                #print("Found display 0")
                #print(f"{unknown[i].value:7b} ")
                #print(f"{mask1.value:7b} ")
                display0 = unknown[i]
                unknown.remove(unknown[i])
            else:
                i += 1
        i = 0
        while i < len(unknown):
            mask1 = SegmentGroup()
            mask1.value = (unknown[i].value & display9.value)

            if mask1.CountFlags() == 4 and unknown[i].flags == 5:
                #print("Found display 2")
                #print(f"{unknown[i].value:7b} ")
                #print(f"{mask1.value:7b} ")
                display2 = unknown[i]
                unknown.remove(unknown[i])
            else:
                i += 1


        self.decoder = [display0, display1, display2, display3, display4, display5, display6, display7, display8, display9]

        self.value = 0
        for i in range(len(self.output)):
            for j in range(len(self.decoder)):
                if self.output[i].value == self.decoder[j].value:
                    self.value += j * int(math.pow(10,len(self.output) - i-1))
    # print decode value as a 7 segment display
    def Print(self):
        displays = [""]*9
        for i in range(len(self.output)):
            for j in range(len(self.decoder)):
                if self.output[i].value == self.decoder[j].value:
                    displayvalue = Display(j)
                    #  aaaaa
                    # b     c
                    # b     c
                    # b     c
                    #  ddddd
                    # e     f
                    # e     f
                    # e     f
                    #  ggggg
                    if displayvalue.name.__contains__("a"):
                        displays[0]+=" #####   "
                    else:
                        displays[0]+="         "
                    
                    if displayvalue.name.__contains__("b"):
                        displays[1]+="#"
                        displays[2]+="#"
                        displays[3]+="#"
                    else:
                        displays[1]+=" "
                        displays[2]+=" "
                        displays[3]+=" "
                    
                    if displayvalue.name.__contains__("c"):
                        displays[1]+="     #  "
                        displays[2]+="     #  "
                        displays[3]+="     #  "
                    else:
                        displays[1]+="        "
                        displays[2]+="        "
                        displays[3]+="        "
                    
                    if displayvalue.name.__contains__("d"):
                        displays[4]+=" #####   "
                    else:
                        displays[4]+="         "

                    if displayvalue.name.__contains__("e"):
                        displays[5]+="#"
                        displays[6]+="#"
                        displays[7]+="#"
                    else:
                        displays[5]+=" "
                        displays[6]+=" "
                        displays[7]+=" "
                    
                    if displayvalue.name.__contains__("f"):
                        displays[5]+="     #  "
                        displays[6]+="     #  "
                        displays[7]+="     #  "
                    else:
                        displays[5]+="        "
                        displays[6]+="        "
                        displays[7]+="        "
                    
                    if displayvalue.name.__contains__("g"):
                        displays[8]+=" #####   "
                    else:
                        displays[8]+="         "
        
        for i in range(len(displays)):
            print(displays[i])

            
    

class Display(Enum):
    abcefg =0   # a b c   e f g //
    cf = 1      #     c     f   //
    acdeg = 2   # a   c d e   g //
    acdfg = 3   # a   c d   f g //
    bcdf = 4    #   b c d   f   //
    abdfg = 5   # a b   d   f g //
    abedfg = 6  # a b   d e f g //
    acf = 7     # a   c     f   //
    abcdefg = 8 # a b c d e f g //
    abcdfg = 9   # a b c d  f g //

def GetSegments(dataSet):
    data = []
    with open(dataSet) as f:
        data = f.readlines()
    

    for i in range(len(data)):
        data[i] = data[i].strip().split('|')
        for j in range(len(data[i])):
            data[i][j] = data[i][j].split(' ')
            data[i][j].remove('')
            for k in range(len(data[i][j])):
                newSegment = SegmentGroup()
                newSegment.Encode(data[i][j][k])
                data[i][j][k] = newSegment
        data[i] = SegmentBoard(data[i][0], data[i][1])

    return data


data = GetSegments('./data/aoc8.txt')

# Part 1
count = 0
for i in range(len(data)):
    for j in range(len(data[i].output)):
        # if count of flags in segment is 2 it displays a 1
        if data[i].output[j].flags == 2:
            count += 1
        if data[i].output[j].flags == 3:
            count += 1
        if data[i].output[j].flags == 4:
            count += 1
        if data[i].output[j].flags == 7:
            count += 1
print()
print(f"Part 1: {count}")

# Part 2
sumvalue = 0
for i in range(len(data)):
    data[i].Decode()
    sumvalue += data[i].value
print(f"Part 2: {sumvalue}")

print("")

for i in range(len(data)):
    data[i].Print()
    print("")



Part 1: 321
Part 2: 1028926

          #####             #####   
#     #  #     #  #     #  #     #  
#     #  #     #  #     #  #     #  
#     #  #     #  #     #  #     #  
 #####    #####    #####    #####   
      #        #        #        #  
      #        #        #        #  
      #        #        #        #  
          #####             #####   

 #####                      #####   
      #        #  #     #        #  
      #        #  #     #        #  
      #        #  #     #        #  
                   #####            
      #        #        #        #  
      #        #        #        #  
      #        #        #        #  
                                    

          #####    #####    #####   
      #        #        #  #        
      #        #        #  #        
      #        #        #  #        
          #####             #####   
      #  #              #        #  
      #  #              #        #  
      #  #              #        #  
      